In [1]:
!pip install psycopg2-binary lxml

In [2]:
import psycopg2
import xml.etree.ElementTree as ET
import os
from lxml import etree

In [3]:
# Database connection params
DB_NAME = "lei"
DB_USER = "postgres"
DB_PASSWORD = "password"
DB_HOST = "db"
DB_PORT = "5432"

conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
)

In [16]:
def find(item, value, ns):
    res = item.find(value, ns)
    if res is not None:
        return res.text
    else:
        return None

In [6]:
XML_FILE = "./20250618-0800-gleif-goldencopy-lei2-golden-copy.xml"
tree = ET.parse(XML_FILE)
root = tree.getroot()

NameError: name 'ns' is not defined

In [20]:
ns = {'lei': 'http://www.gleif.org/data/schema/leidata/2016'}
records = tree.find("lei:LEIRecords", ns)

with conn.cursor() as cur:
    # Example: Suppose XML structure is <items><item><name>...</name><price>...</price></item></items>
    for item in records.findall("lei:LEIRecord" ,ns):
        lei = item.find("lei:LEI", ns).text
        name = find(item, "lei:Entity/lei:LegalName", ns)
        first_address_line = find(item, "lei:Entity/lei:LegalAddress/lei:FirstAddressLine", ns)
        address_line = find(item, "lei:Entity/lei:LegalAddress/lei:FirstAddressLine", ns)
        address_city = find(item, "lei:Entity/lei:LegalAddress/lei:City", ns)
        address_region = find(item, "lei:Entity/lei:LegalAddress/lei:Region", ns)
        address_country = find(item, "lei:Entity/lei:LegalAddress/lei:Country", ns)      
        
        # Insert into your table
        cur.execute(
            "INSERT INTO LegalEntities (lei, address_line, address_city, address_region, address_country) VALUES (%s, %s, %s, %s, %s)",
            (lei, address_line, address_city, address_region, address_country)
        )

        cur.execute(
            "INSERT INTO Names (lei,name) VALUES (%s, %s)",
            (lei, name)
        )
        other_names = item.find("lei:Entity/lei:OtherEntityNames", ns)
        if other_names is not None:
            for names_item in other_names.findall("lei:OtherEntityName"):
                names = names_item.text
                cur.execute(
                    "INSERT INTO Names (lei,name) VALUES (%s, %s)",
                    (lei, names)
                )
conn.commit()

/tmp/ipykernel_629/3123213431.py:26: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if other_names:


In [26]:
with conn.cursor() as cur:
    for item in records.findall("lei:LEIRecord" ,ns):
        lei = item.find("lei:LEI", ns).text
        other_names = item.find("lei:Entity/lei:OtherEntityNames", ns)
        if other_names is not None:
            for names_item in other_names.findall("lei:OtherEntityName", ns):
                names = names_item.text
                cur.execute(
                    "INSERT INTO Names (lei,name) VALUES (%s, %s)",
                    (lei, names)
                )
conn.commit()

In [ ]:
conn.close()